<a href="https://colab.research.google.com/github/nyanta012/chatgpt_api_practice/blob/main/section5/section5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Colaboratoryのショートカットキー
1. **現在のセルを実行し、次のセルを選択**: `Shift + Enter`
2. **コードセルをテキストセルにする**: `Ctrl + M M`
3. **コードセルを上に追加**: `Ctrl + M A`
4. **コードセルを下に追加**: `Ctrl + M B`
5. **セルの削除**: `Ctrl + M D`

# API KEYの設定

In [1]:
%%capture
!pip install openai

In [2]:
import openai
import getpass
from typing import List, Dict, Tuple

apikey = getpass.getpass(prompt="OpenAIのAPIキーを入力してください ")

openai.api_key = apikey

OpenAIのAPIキーを入力してください ··········


# プロンプトを入力して返答を返す関数を定義

In [3]:
def get_chatgpt_response(
    user_input: str,
    template: str,
    model: str = "gpt-3.5-turbo-0613",
    temperature: float = 0,
    max_tokens: int = 500,
):
    """
    ChatGPTに対して対話を投げかけ、返答を取得する
    """
    prompt = template.format(user_input=user_input)
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
    )
    return response.choices[0].message["content"]

In [ ]:
PROMPT_TEMPLATE = """
    下記の文章を英語にしてください。

    {user_input}

    """

In [ ]:
PROMPT_TEMPLATE = """
    下記の文章を関西弁にしてください。

    {user_input}

    """

In [ ]:
print(get_chatgpt_response("がむしゃらにやれ！", PROMPT_TEMPLATE))

がんばりやー！


# 問い合わせ分類

**問い合わせが以下のどのカテゴリーか分類させよう**
  - 製品に対する問い合わせ
  - 注文に対する問い合わせ
  - 配送に対する問い合わせ

まずは、プロンプトのテンプレートを定義

In [ ]:
PROMPT_TEMPLATE = """
    下記の####で区切られた顧客からの問い合わせがあります。
    問い合わせ内容が下記のどのカテゴリーに当てはまるか考えてください。
    下記に当てはまらない場合は、「該当するカテゴリーがありません」と返答してください。

    カテゴリー：
     製品に対する問い合わせ
     注文に対する問い合わせ
     配送に対する問い合わせ

    ####

    {user_input}

    ####

    出力は必ず下記のどれかを出力してください
    製品に対する問い合わせ, 注文に対する問い合わせ, 配送に対する問い合わせ
    """

In [ ]:
get_chatgpt_response("本日購入した場合、最短で納期はいつですか？", PROMPT_TEMPLATE)

'配送に対する問い合わせ'

In [ ]:
get_chatgpt_response("購入をキャンセルしたいです", PROMPT_TEMPLATE)

'注文に対する問い合わせ'

In [ ]:
get_chatgpt_response("どのような商品がありますか？", PROMPT_TEMPLATE)

'製品に対する問い合わせ'

In [ ]:
get_chatgpt_response("猫を飼いたいです", PROMPT_TEMPLATE)

'該当するカテゴリーがありません。'

# 問い合わせの詳細カテゴリー

問い合わせの内容により、以下のカテゴリーに分類します。

## 製品に対する問い合わせ
  - **スペック**
  - **値段**
  - **購入方法**

## 注文に対する問い合わせ
  - **注文状況**
  - **注文変更**
  - **支払い方法**

## 配送に対する問い合わせ
  - **配送ステータス**
  - **配送オプション**
  - **配送問題**


In [13]:
PROMPT_TEMPLATE = """
    下記の####で区切られた顧客からの問い合わせがあります。
    問い合わせ内容を最初に第一カテゴリーで分けて、その後、第一カテゴリーに紐づく第二カテゴリーに分けてください。
    カテゴリーに当てはまるものが無い場合「該当するカテゴリーがありません」と返答してください。

    第一カテゴリー：
      製品, 注文, 配送

    第二カテゴリー：
     ・製品の場合
       スペック, 値段, 購入方法
     ・注文の場合
       注文状況, 注文変更, 支払い方法
      ・配送の場合
       配送ステータス, 配送オプション, 配送問題

    ####

    {user_input}

    ####

    出力は下記の形式で出力してください
      第一カテゴリー:<第一カテゴリーのどれか>
      第二カテゴリー:<第二カテゴリーのどれか>
    """

In [14]:
response = get_chatgpt_response("私のパッケージは出荷されましたか？", PROMPT_TEMPLATE)
print(response)

第一カテゴリー: 配送
第二カテゴリー: 配送ステータス


In [15]:
response = get_chatgpt_response("到着日時を変更して欲しいです", PROMPT_TEMPLATE)
print(response)

第一カテゴリー: 配送
第二カテゴリー: 配送問題


In [16]:
response = get_chatgpt_response("商品が壊れました", PROMPT_TEMPLATE)
print(response)

第一カテゴリー: 製品
第二カテゴリー: 該当するカテゴリーがありません


In [ ]:
response = get_chatgpt_response("到着日時を変更して欲しいです", PROMPT_TEMPLATE)
print(response)

第一カテゴリー:配送
      第二カテゴリー:配送オプション


In [ ]:
print(response.replace(" ", ""))

第一カテゴリー:配送
第二カテゴリー:配送オプション


# JSON形式で出力させてみよう

In [4]:
PROMPT_TEMPLATE = """
    下記の####で区切られた顧客からの問い合わせがあります。
    問い合わせ内容を最初に第一カテゴリーで分けて、その後、第一カテゴリーにそれぞれ紐づく第二カテゴリーに分けてください。
    カテゴリーに当てはまるものが無い場合「該当するカテゴリーがありません」と返答してください。

    第一カテゴリー：
      製品, 注文, 配送

    第二カテゴリー：
     ・製品の場合
       スペック, 値段, 購入方法
     ・注文の場合
       注文状況, 注文変更, 支払い方法
      ・配送の場合
       配送ステータス, 配送オプション, 配送問題

    ####

    {user_input}

    ####

    出力は必ず、JSON形式(key=<第一カテゴリーのどれか>, value=<第二カテゴリーのどれか>)で出力してください。
    例： {{"key": "製品", "value": "値段"}}
    該当するカテゴリーが無い場合key,value両方とも 該当なし としてください
    """

In [5]:
print(get_chatgpt_response("このスマートフォンの最新の価格は何ですか？", PROMPT_TEMPLATE))

{"key": "製品", "value": "値段"}


# 問い合わせ内容が膨大にある
  - "このスマートフォンのプロセッサーは何ですか？"
  - "このパソコンのRAMの容量はどれくらいですか？"
  - "このカメラの最大シャッタースピードは何ですか？"
  - "このヘッドフォンの周波数レンジは何ですか？"
  - "この自転車のフレームは何で作られていますか？"
  - "このスマートフォンの最新の価格は何ですか？"
  - "このゲーム機の割引価格はありますか？"
  - "この製品の月々の費用はどれくらいですか？"
  - "このサブスクリプションサービスの年間費用は何ですか？"
  - "この洗濯機には何年間の保証が含まれていますか？そのコストはどれくらいですか？"
  - "この製品はオンラインで注文することができますか？"
  - "この製品を店舗で購入することは可能ですか？"
  - "この製品は分割払いで購入できますか？"
  - "この製品は海外から注文できますか？"
  - "この製品を予約注文することは可能ですか？"
  - "私の注文はすでに確認されましたか？"
  - "私の注文はまだ出荷されていませんか？"
  - "注文した商品の在庫状況を教えてください。"
  - "注文の進行状況を確認するにはどうすればよいですか？"
  - "私の注文はキャンセルされましたか？"
  - "私の注文の配送先住所を変更することは可能ですか？"
  - "注文した商品の色やサイズを変更することは可能ですか？"
  - "注文した商品を別のものに変更することは可能ですか？"
  - "注文の数量を増やすことは可能ですか？"  
  - "注文した商品をキャンセルすることは可能ですか？"
  - "注文の支払いにクレジットカードは使用できますか？"
  - "注文の支払いにデビットカードは使用できますか？"
  - "注文の支払いにPayPalは使用できますか？"
  - "注文の支払いにApple PayやGoogle Payは使用できますか？"
  - "注文の支払いにビットコインは使用できますか？"
  - "私のパッケージは出荷されましたか？"
  - "私のパッケージの追跡番号を教えてください。"
  - "私のパッケージはいつ到着予定ですか？"
  - "私のパッケージが遅延しています。なぜですか？"
  - "私のパッケージは配送中に失われたようです。どうすればいいですか？"
  - "エクスプレス配送は可能ですか？"
  - "特定の日時に配送することはできますか？"
  - "配送先を注文後に変更することは可能ですか？"
  - "商品の配送はどの運送会社を利用していますか？"
  - "私の地域への配送は可能ですか？"
  - "配送された商品が損傷していました。どうすればいいですか？"
  - "配送された商品が間違っていました。どうすればいいですか？"
  - "パッケージが届かない場合、どうすればいいですか？"
  - "配達員が不在票を残していました。どうすればいいですか？"
  - "パッケージが開封されていました。どうすればいいですか？"

In [6]:
questions = [
    "このカメラの最大シャッタースピードは何ですか？",
    "私のパッケージの追跡番号を教えてください。",
    "エクスプレス配送は可能ですか？",
    "特定の日時に配送することはできますか？",
    "配送された商品が損傷していました。どうすればいいですか？",
    "注文の支払いにApple PayやGoogle Payは使用できますか？",
    "私のパッケージはいつ到着予定ですか？",
    "注文の支払いにクレジットカードは使用できますか？",
    "私の注文はまだ出荷されていませんか？",
    "この製品はオンラインで注文することができますか？",
    "私のパッケージが遅延しています。なぜですか？",
    "このスマートフォンの最新の価格は何ですか？",
    "注文した商品をキャンセルすることは可能ですか？",
    "商品の配送はどの運送会社を利用していますか？",
    "このサブスクリプションサービスの年間費用は何ですか？",
    "このヘッドフォンの周波数レンジは何ですか？",
    "私の地域への配送は可能ですか？",
    "注文の支払いにビットコインは使用できますか？",
    "配送された商品が間違っていました。どうすればいいですか？",
    "この製品を店舗で購入することは可能ですか？",
    "パッケージが届かない場合、どうすればいいですか？",
    "私の注文はキャンセルされましたか？",
    "このパソコンのRAMの容量はどれくらいですか？",
    "注文の支払いにデビットカードは使用できますか？",
    "私のパッケージは出荷されましたか？",
    "注文の数量を増やすことは可能ですか？",
    "私の注文はすでに確認されましたか？",
    "注文の進行状況を確認するにはどうすればよいですか？",
    "配送先を注文後に変更することは可能ですか？",
    "パッケージが開封されていました。どうすればいいですか？",
    "配達員が不在票を残していました。どうすればいいですか？",
    "私のパッケージは配送中に失われたようです。どうすればいいですか？",
    "この自転車のフレームは何で作られていますか？",
    "この製品を予約注文することは可能ですか？",
    "注文した商品の色やサイズを変更することは可能ですか？",
    "この製品の月々の費用はどれくらいですか？",
    "このスマートフォンのプロセッサーは何ですか？",
    "この製品は分割払いで購入できますか？",
    "この製品は海外から注文できますか？",
    "注文した商品の在庫状況を教えてください。",
    "このゲーム機の割引価格はありますか？",
    "私の注文の配送先住所を変更することは可能ですか？",
    "注文した商品を別のものに変更することは可能ですか？",
    "この洗濯機には何年間の保証が含まれていますか？そのコストはどれくらいですか？",
    "注文の支払いにPayPalは使用できますか？",
]

In [7]:
from tqdm import tqdm
import json


def get_responses_and_check_json(questions: List[str]):
    """
    ChatGPTからの応答を取得し、それらがJSON形式であることを確認する
    """
    list_json = []

    def json_checker(response):
        try:
            return json.loads(response.strip())
        except json.JSONDecodeError:
            print(f"No json format... {response.strip()}")

    for question in tqdm(questions):
        response = get_chatgpt_response(question, PROMPT_TEMPLATE)
        list_json.append(json_checker(response))

    return list_json

In [8]:
list_json = get_responses_and_check_json(questions)

100%|██████████| 45/45 [00:49<00:00,  1.11s/it]


In [9]:
import pandas as pd

df = pd.DataFrame(list_json)
df["question"] = questions
df.columns = ["category_1", "category_2", "question"]

In [10]:
df

,category_1,category_2,question
0,製品,スペック,このカメラの最大シャッタースピードは何ですか？
1,配送,配送ステータス,私のパッケージの追跡番号を教えてください。
2,配送,配送オプション,エクスプレス配送は可能ですか？
3,配送,配送オプション,特定の日時に配送することはできますか？
4,配送,配送問題,配送された商品が損傷していました。どうすればいいですか？
5,注文,支払い方法,注文の支払いにApple PayやGoogle Payは使用できますか？
6,配送,配送ステータス,私のパッケージはいつ到着予定ですか？
7,注文,支払い方法,注文の支払いにクレジットカードは使用できますか？
8,注文,注文状況,私の注文はまだ出荷されていませんか？
9,製品,購入方法,この製品はオンラインで注文することができますか？


In [11]:
df["category_1"].value_counts(normalize=True) * 100

注文    37.777778
製品    33.333333
配送    28.888889
Name: category_1, dtype: float64

In [12]:
df["category_2"].value_counts(normalize=True) * 100

スペック               13.333333
配送問題               13.333333
注文変更               13.333333
該当するカテゴリーがありません    13.333333
支払い方法              11.111111
注文状況               11.111111
購入方法                8.888889
配送ステータス             6.666667
配送オプション             4.444444
値段                  4.444444
Name: category_2, dtype: float64

# 各カテゴリーに対応した返答文の生成

問い合わせカテゴリーごとに、適切な問い合わせ先を示す返答文を作成しましょう。

## シチュエーション

具体的には、ユーザーからの問い合わせを適切に分類し、その問い合わせの内容に応じた連絡先をユーザーに教える返答文を自動生成します。


In [ ]:
def get_chatgpt_response(
    user_input: str,
    category_1: str,
    category_2: str,
    template: str,
    model: str = "gpt-3.5-turbo",
    temperature: float = 0,
    max_tokens: int = 500,
):
    """
    ChatGPTに対して対話を投げかけ、返答を取得する
    """
    prompt = template.format(
        user_input=user_input, category_1=category_1, category_2=category_2
    )
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
    )
    return response.choices[0].message["content"]

In [ ]:
PROMPT_TEMPLATE = """
    あなたはカスタマーセンターの従業員です。
    あなたの仕事は顧客の問い合わせに対して、適切な部門の連絡先を教えることです。
    顧客の問い合わせ内容とカテゴリ－は下記の####で区切られた内容です。

    ####
    {user_input}

    第一カテゴリー:{category_1}
    第二カテゴリー:{category_2}
    ####

    下記ステップに従って、メールを作成してください。

    1. 第一カテゴリーと第二カテゴリーに応じてメールアドレスを下記から抽出してください。
        製品：スペック は product_spec@example.com
        製品：値段 は product_price@example.com
        製品：購入方法 は product_purchase_method@example.com
        注文：注文状況 は order_status@example.com
        注文：注文変更 は order_change@example.com
        注文：支払い方法 は order_payment_method@example.com
        配送：配送ステータス は delivery_status@example.com
        配送：配送オプション は delivery_option@example.com
        配送：配送問題 は delivery_issue@example.com

    2. メール文章を作成
      ユーザーからの問い合わせ内容に対して1で抽出したメールアドレスに送信するようにお願いする文章を書いてください。
        形式：
          適切な件名

          文章
    """

In [ ]:
for values in df.values[:10]:
    print("")
    print("#" * 100)
    category_1, category_2, user_question = values[0], values[1], values[2]
    print(f"問い合わせ内容:{user_question}\n")
    response = get_chatgpt_response(
        user_input=user_question,
        category_1=category_1,
        category_2=category_2,
        template=PROMPT_TEMPLATE,
    )
    print(response)



####################################################################################################
問い合わせ内容:このカメラの最大シャッタースピードは何ですか？

件名：カメラの最大シャッタースピードについてのお問い合わせ

お客様からのお問い合わせに対し、適切な部署に連絡いたします。

このお問い合わせは、製品のスペックに関するものであり、以下のメールアドレスにお問い合わせいただけます。

product_spec@example.com

ご不明点がございましたら、お気軽にお問い合わせください。

よろしくお願いいたします。

####################################################################################################
問い合わせ内容:私のパッケージの追跡番号を教えてください。

件名：配送ステータスに関するお問い合わせ

お客様

この度は弊社にお問い合わせいただき、誠にありがとうございます。
お客様からのお問い合わせにつきまして、配送ステータスに関するお問い合わせであることを確認いたしました。

お手数をおかけいたしますが、下記のメールアドレスにお問い合わせ内容をご連絡いただけますでしょうか。
配送：配送ステータス は delivery_status@example.com

お客様のご理解とご協力に心より感謝申し上げます。

敬具
カスタマーセンター

####################################################################################################
問い合わせ内容:エクスプレス配送は可能ですか？

件名：エクスプレス配送についてのお問い合わせ

お客様

いつも弊社をご利用いただきありがとうございます。お問い合わせいただきましたエクスプレス配送については、配送オプションのカテゴリーに該当いたします。

配送オプションに関するお問い合わせは、delivery_option@example.com までご連絡ください。専門のスタッフが迅速に対応いた